In [1]:
%load_ext autotime
import geopandas as gpd
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
tqdm.pandas()
from glob import glob
import os
from shapely import line_interpolate_point, Point
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
import plotly.express as px
pd.options.plotting.backend = "plotly"
files = sorted(glob("Profiles_T3_T7/*"))
files

['Profiles_T3_T7/POETTO_2017_01_25_T7.csv',
 'Profiles_T3_T7/POETTO_2017_04_06_T3.CSV',
 'Profiles_T3_T7/POETTO_2017_04_06_T7.CSV',
 'Profiles_T3_T7/POETTO_2018_10_09_T3.CSV',
 'Profiles_T3_T7/POETTO_2018_10_09_T7.CSV',
 'Profiles_T3_T7/POETTO_2018_10_09_T7_Modified.txt',
 'Profiles_T3_T7/POETTO_2018_10_09_T7_Modifiedd.txt',
 'Profiles_T3_T7/POETTO_2018_10_19_T3.CSV',
 'Profiles_T3_T7/POETTO_2018_10_19_T7.CSV',
 'Profiles_T3_T7/POETTO_2019_05_21_T3.CSV',
 'Profiles_T3_T7/POETTO_2019_05_21_T7.CSV',
 'Profiles_T3_T7/POETTO_2020_01_20_T3.CSV',
 'Profiles_T3_T7/POETTO_2020_01_20_T7.CSV',
 'Profiles_T3_T7/POETTO_2020_01_27_T3.CSV',
 'Profiles_T3_T7/POETTO_2020_01_27_T7.CSV',
 'Profiles_T3_T7/POETTO_2020_03_28_T3.csv',
 'Profiles_T3_T7/POETTO_2020_03_28_T7.csv',
 'Profiles_T3_T7/POETTO_2020_05_04_T3.CSV',
 'Profiles_T3_T7/POETTO_2020_05_04_T7.CSV',
 'Profiles_T3_T7/POETTO_2022_11_21_NEWEST.csv',
 'Profiles_T3_T7/POETTO_2022_11_21_NEWEST_v2.csv',
 'Profiles_T3_T7/POETTO_2022_11_21_T3.CSV',
 '

In [2]:
transects = gpd.read_file("transects_extended.geojson").set_index("id")
transects

,site_id,orientation,along_dist,along_dist_norm,beach_slope,cil,ciu,trend,n_points,n_points_nonan,r2_score,mae,mse,rmse,intercept,ERODIBILITY,geometry
id,,,,,,,,,,,,,,,,,
aus0001-0000,aus0001,104.347648,0.000000,0.000000,0.085,0.0545,0.2000,-1.456305,654.0,342.0,0.131951,28.721214,1321.059410,36.346381,179.407277,None,"LINESTRING (153.26555 -24.7007, 153.26938 -24...."
aus0001-0001,aus0001,93.495734,98.408334,0.002935,0.050,0.0387,0.0640,-0.833581,654.0,473.0,0.052783,26.029482,1092.336862,33.050520,210.629160,None,"LINESTRING (153.26525 -24.7019, 153.2692 -24.7..."
aus0001-0002,aus0001,82.069341,198.408334,0.005918,0.050,0.0428,0.0647,-0.282209,654.0,491.0,0.008220,22.896464,838.997233,28.965449,200.166307,None,"LINESTRING (153.26539 -24.70316, 153.26931 -24..."
aus0001-0003,aus0001,81.192757,298.402523,0.008900,0.055,0.0480,0.0659,0.089340,654.0,502.0,0.000987,20.770475,691.179232,26.290288,185.295473,None,"LINESTRING (153.26555 -24.70408, 153.26945 -24..."
aus0001-0004,aus0001,81.065473,398.402523,0.011882,0.075,0.0614,0.0922,0.361437,654.0,508.0,0.017461,19.528839,637.676213,25.252252,169.027861,None,"LINESTRING (153.2657 -24.70497, 153.26961 -24...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sar2538-0004,sar2538,NaN,NaN,NaN,NaN,NaN,NaN,-0.072765,1220.0,1220.0,0.159922,1.468474,3.900855,1.975058,244.639331,Low,"LINESTRING (9.33568 41.29334, 9.33294 41.29629)"
sar2539-0000,sar2539,NaN,NaN,NaN,NaN,NaN,NaN,0.094468,388.0,378.0,0.023321,2.795524,24.861725,4.986153,214.008472,Low,"LINESTRING (9.40022 41.3002, 9.40172 41.30362)"
sar2539-0001,sar2539,NaN,NaN,NaN,NaN,NaN,NaN,0.161493,388.0,376.0,0.058879,3.099783,27.664877,5.259741,191.216499,Low,"LINESTRING (9.39917 41.3019, 9.4035 41.30039)"


In [3]:
intersects = pd.read_csv("data/sar0941/transect_time_series.csv")
intersects.dates = pd.to_datetime(intersects.dates)
transect_id = 'sar0941-0000'
intersects[transect_id] = gpd.GeoSeries(line_interpolate_point(transects.geometry.to_crs(3003)[transect_id], intersects[transect_id]), crs=3003)

In [4]:
intersects["timediff"] = (intersects.dates - pd.to_datetime("2017-01-25 00:00:00+00:00")).abs()
intersects.sort_values("timediff")

,dates,satname,sar0941-0000,sar0941-0001,sar0941-0002,sar0941-0003,sar0941-0004,sar0941-0005,sar0941-0006,sar0941-0007,timediff
767,2017-01-25 10:06:38+00:00,L8,POINT (1515236.007 4340431.221),202.61,200.94,195.35,204.47,202.22,207.47,200.08,0 days 10:06:38
768,2017-01-26 10:02:22+00:00,L7,POINT (1515236.588 4340430.275),203.36,206.11,204.49,NaN,NaN,-55.21,NaN,1 days 10:02:22
766,2017-01-09 10:06:43+00:00,L8,POINT (1515240.141 4340424.489),209.10,203.17,201.10,208.10,211.57,211.66,204.62,15 days 13:53:17
769,2017-02-11 10:02:16+00:00,L7,None,NaN,192.32,189.09,NaN,NaN,NaN,196.77,17 days 10:02:16
765,2017-01-02 10:00:33+00:00,L8,POINT (1515240.225 4340424.352),212.48,206.36,199.93,208.57,211.48,216.35,207.86,22 days 13:59:27
...,...,...,...,...,...,...,...,...,...,...,...
4,1984-06-07 09:34:42+00:00,L5,POINT (1515224.856 4340449.381),181.85,175.61,185.80,189.06,192.88,200.27,188.61,11919 days 14:25:18
3,1984-05-31 09:28:23+00:00,L5,POINT (1515225.814 4340447.821),181.54,176.86,179.99,182.05,188.07,188.85,181.16,11926 days 14:31:37
2,1984-05-06 09:33:51+00:00,L5,POINT (1515226.431 4340446.816),179.34,175.95,175.88,179.33,183.80,185.50,182.47,11951 days 14:26:09
1,1984-04-29 09:27:29+00:00,L5,POINT (1515220.801 4340455.985),172.37,172.57,176.45,185.43,182.83,190.28,170.60,11958 days 14:32:31


In [5]:
df = pd.read_csv(files[0], sep=";", names=["lat", "lng", "elevation"])
df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lng, df.lat), crs=32632)
m = df.explore("elevation")
gpd.GeoSeries(intersects[transect_id])[intersects.dates == "2017-01-25 10:06:38+00:00"].explore(m=m)

In [70]:
results = []
profiles = []

def interpolate_zero_elevation(df: pd.DataFrame) -> tuple | None:
    """
    Finds the two data points closest to elevation 0 (one positive, one negative)
    and interpolates their latitude and longitude to estimate the coordinates
    at elevation 0.

    Args:
        df: A pandas DataFrame with columns 'lat', 'lng', and 'elevation'.
            Assumes valid data and necessary columns exist.

    Returns:
        A tuple containing the interpolated (latitude, longitude) at elevation 0,
        or None if interpolation is not possible.
    """
    # Optional: Handle exact zero first for efficiency
    exact_zero = df.loc[df['elevation'] == 0]
    if not exact_zero.empty:
        return tuple(exact_zero[['lat', 'lng']].iloc[0])

    try:
        # Find indices of the closest points above and below zero
        idx_pos = df.loc[df['elevation'] > 0, 'elevation'].idxmin()
        idx_neg = df.loc[df['elevation'] < 0, 'elevation'].idxmax()

        # Get the two points using the indices
        p_neg = df.loc[idx_neg]
        p_pos = df.loc[idx_pos]

    except ValueError:
        # Occurs if idxmin/idxmax fails (e.g., no positive or no negative elevations)
        return None

    # Unpack values for interpolation
    e1, lat1, lng1 = p_neg['elevation'], p_neg['lat'], p_neg['lng']
    e2, lat2, lng2 = p_pos['elevation'], p_pos['lat'], p_pos['lng']

    # Calculate interpolation factor and interpolate lat/lng
    # (e2 - e1) cannot be zero here because one is < 0 and one is > 0
    factor = -e1 / (e2 - e1)
    interp_lat = lat1 + factor * (lat2 - lat1)
    interp_lng = lng1 + factor * (lng2 - lng1)

    return (interp_lat, interp_lng)

for f in files:
    date = f[22:32]
    site = f[33:35]
    try:
        df = pd.read_csv(f, sep=";", names=["lat", "lng", "elevation"], index_col=False)
        df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lng, df.lat), crs=32632)
        zero_lat, zero_lng = interpolate_zero_elevation(df)

        if date == "2020_03_28" and site == "T3":
            df["distance"] = df.distance(df.geometry.iloc[-1])
            zero_dist_from_origin = Point(zero_lng, zero_lat).distance(df.geometry.iloc[-1])
        else:
            df["distance"] = df.distance(df.geometry[0])
            zero_dist_from_origin = Point(zero_lng, zero_lat).distance(df.geometry[0])
        df["site"] = site
        df["date"] = date
        profiles.append(df)
        results.append({
            "date": date,
            "site": site,
            "lat": zero_lat,
            "lng": zero_lng,
            "origin_lat": df.lat[0],
            "origin_lng": df.lng[0],
            "dist_from_origin": zero_dist_from_origin,
        })
    except Exception as e:
        pass
        #print(f"Error processing {f}: {e}")
df = pd.DataFrame(results)
df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.origin_lng, df.origin_lat), crs=32632)
df.date = pd.to_datetime(df.date, format="%Y_%m_%d")
display(df)
m = df.explore("date")
profiles = pd.concat(profiles)
gpd.GeoSeries(intersects[transect_id]).explore(m=m, name="CoastSat")
gpd.GeoSeries(transects.geometry["sar0939-0000"]).explore(m=m, name="transect")

/tmp/ipykernel_159161/2174370178.py:52: ParserWarning:

Length of header or names does not match length of data. This leads to a loss of data with index_col=False.

/tmp/ipykernel_159161/2174370178.py:52: ParserWarning:

Length of header or names does not match length of data. This leads to a loss of data with index_col=False.

/tmp/ipykernel_159161/2174370178.py:52: ParserWarning:

Length of header or names does not match length of data. This leads to a loss of data with index_col=False.

/tmp/ipykernel_159161/2174370178.py:52: ParserWarning:

Length of header or names does not match length of data. This leads to a loss of data with index_col=False.

/tmp/ipykernel_159161/2174370178.py:52: ParserWarning:

Length of header or names does not match length of data. This leads to a loss of data with index_col=False.

/tmp/ipykernel_159161/2174370178.py:52: ParserWarning:

Length of header or names does not match length of data. This leads to a loss of data with index_col=False.

/tmp/ipyke

,date,site,lat,lng,origin_lat,origin_lng,dist_from_origin,geometry
0,2017-01-25,T7,4.340424e+06,5.151973e+05,4.340509e+06,5.151222e+05,113.657573,POINT (515122.167 4340508.834)
1,2017-04-06,T3,4.339303e+06,5.141540e+05,4.339361e+06,5.140721e+05,99.920179,POINT (514072.113 4339360.512)
2,2017-04-06,T7,4.340489e+06,5.152692e+05,4.340568e+06,5.152032e+05,102.916636,POINT (515203.246 4340568.068)
3,2018-10-09,T3,4.339309e+06,5.141474e+05,4.339360e+06,5.140721e+05,91.277891,POINT (514072.1 4339360.293)
4,2018-10-09,T7,4.340426e+06,5.151952e+05,4.340508e+06,5.151228e+05,109.484944,POINT (515122.812 4340508.414)
5,2018-10-19,T3,4.340427e+06,5.151934e+05,4.340509e+06,5.151234e+05,107.668513,POINT (515123.386 4340509.211)
6,2018-10-19,T7,4.340427e+06,5.151934e+05,4.340509e+06,5.151234e+05,107.668513,POINT (515123.386 4340509.211)
7,2019-05-21,T7,4.340428e+06,5.151927e+05,4.340500e+06,5.151305e+05,94.671565,POINT (515130.492 4340499.69)
8,2020-01-20,T3,4.339322e+06,5.141479e+05,4.339360e+06,5.140717e+05,85.276950,POINT (514071.697 4339360.126)
9,2020-01-20,T7,4.340427e+06,5.151918e+05,4.340510e+06,5.151242e+05,106.644953,POINT (515124.157 4340509.72)


In [28]:
px.line(profiles[profiles.site == "T3"], x="distance", y="elevation", color="date", title="Elevation profiles at T3", height=600, markers=True)

In [29]:
px.line(profiles[profiles.site == "T7"], x="distance", y="elevation", color="date", title="Elevation profiles at T7", height=600, markers=True)

In [30]:
transects.to_crs(df.crs).distance(df.geometry[0]).idxmin(), transects.to_crs(df.crs).distance(df.geometry[1]).idxmin()

/home/nyou045/CoastSat/.venv/lib/python3.12/site-packages/shapely/measurement.py:72: RuntimeWarning:

invalid value encountered in distance

/home/nyou045/CoastSat/.venv/lib/python3.12/site-packages/shapely/measurement.py:72: RuntimeWarning:

invalid value encountered in distance



('sar0941-0000', 'sar0939-0000')

In [31]:
T7_offset = Point(df.origin_lng[0], df.origin_lat[0]).distance(Point(transects.geometry.to_crs(df.crs)["sar0941-0000"].coords[0]))
T7_offset

87.98933048805033

In [32]:
T3_offset = Point(df.origin_lng[1], df.origin_lat[1]).distance(Point(transects.geometry.to_crs(df.crs)["sar0939-0000"].coords[0]))
T3_offset

101.37974964880803

In [49]:
df.index = df.date
import plotly.graph_objects as go
fig = go.Figure()
# Create and style traces
fig.add_traces(df.dist_from_origin[(df.site == "T7")&(df["dist_from_origin"]>60)].plot(title="T7 intersects vs CoastSat sar0941-0000", markers=True).data)
cs = pd.read_csv("data/sar0941/transect_time_series_smoothed.csv")
cs.dates = pd.to_datetime(cs.dates)
cs = cs[cs.dates.between("2017-01-25", "2025-01-20")]
cs.set_index("dates", inplace=True)
fig.add_traces((cs["sar0941-0000"] - T7_offset).plot(markers=True).data)

In [64]:
import plotly.graph_objects as go
fig = go.Figure()
# Create and style traces
fig.add_traces(df.dist_from_origin[df.site == "T3"].plot(title="T3 intersects vs CoastSat sar0939-0000", kind="scatter").data)
cs = pd.read_csv("data/sar0939/transect_time_series_smoothed.csv")
cs.dates = pd.to_datetime(cs.dates)
cs = cs[cs.dates.between("2017-01-25", "2025-01-20")]
cs.set_index("dates", inplace=True)
fig.add_traces((cs["sar0939-0000"] - T3_offset).plot(markers=True, color_discrete_sequence=['red']).data)